# Colab Environment:


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
PATH = "/content/drive/MyDrive/공모전/Barksa"
file_path = "/content/drive/MyDrive/공모전/Barksa/preprocessed_names.csv"
df = pd.read_csv(file_path)

In [ ]:
df

# FastText: https://fasttext.cc/


### Why FastText?: It can embed out-of-vocabulary words.


In [ ]:
os.chdir(PATH)
!pwd

/content/drive/MyDrive/공모전/Barksa


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText

Cloning into 'fastText'...
remote: Enumerating objects: 3986, done.
remote: Counting objects: 100% (1014/1014), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 3986 (delta 888), reused 854 (delta 830), pack-reused 2972
Receiving objects: 100% (3986/3986), 8.29 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (2526/2526), done.
Updating files: 100% (529/529), done.


In [ ]:
os.chdir(PATH + "/fastText")

In [ ]:
!make

c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -

In [ ]:
!sudo pip install .

Processing /content/drive/MyDrive/공모전/Barksa/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4214523 sha256=41b2c18841496298ee92b44f0af693dac4f6d799d504aeaf13e3748b8103c450
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqyddj_0/wheels/71/b7/be/3dfddec8fe70d06d36d91db1a762cf1c7a3c860569b0c199da
Successfully built fasttext


In [ ]:
from gensim.models import FastText
import fasttext
import fasttext.util

fasttext.util.download_model("en", if_exists="ignore")
ft = fasttext.load_model("cc.en.300.bin")

In [ ]:
ft.get_nearest_neighbors("environment")

[(0.7772931456565857, 'enviroment'),
 (0.7613595128059387, 'environments'),
 (0.7612473964691162, 'evironment'),
 (0.7547367215156555, 'envrionment'),
 (0.7344476580619812, 'envirnoment'),
 (0.721310019493103, 'environment.The'),
 (0.7144476771354675, 'environemnt'),
 (0.701413631439209, 'environement'),
 (0.6973177790641785, 'enviorment'),
 (0.693863034248352, 'environment.What')]

In [ ]:
def cos_sim(a, b):
    try:
        x = ft.get_word_vector(a)
    except:
        return 0
    try:
        y = ft.get_word_vector(b)
    except:
        return 0
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
def best_n_adj(word, n, adj_list):
    Scores = []
    for i in range(len(adj_list)):
        Scores.append(cos_sim(word, adj_list[i]))
    sorted_Scores = sorted(Scores, reverse=True)
    ranks = [Scores.index(i) for i in sorted_Scores]
    adj_return = []
    for i in range(n):
        adj_return.append(adj_list[ranks[i]])
    return adj_return

In [ ]:
adj_list = [
    "Playful",
    "Gentle",
    "Majestic",
    "Curious",
    "Loyal",
    "Witty",
    "Mysterious",
    "Adventurous",
    "Elegant",
    "Charming",
]

In [ ]:
best_n_adj("Cho Fun", 3, adj_list)

['Witty', 'Curious', 'Majestic']

In [ ]:
name_list = df["Name"]

In [ ]:
name_list = list(name_list)

In [ ]:
name_list

In [ ]:
L = []
for name in name_list:
    L.append([name] + best_n_adj(name, 3, adj_list))

In [ ]:
len(L)

38597

In [ ]:
L

In [ ]:
df_name = pd.DataFrame(L)

In [ ]:
df_name

,0,1,2,3
0,Snowflake,Playful,Charming,Elegant
1,London,Majestic,Mysterious,Playful
2,Gigi,Charming,Majestic,Playful
3,Tintin,Mysterious,Curious,Charming
4,Johnny,Gentle,Witty,Loyal
...,...,...,...,...
38592,Mia,Charming,Playful,Adventurous
38593,Zeus,Majestic,Gentle,Playful
38594,Dalia,Witty,Playful,Gentle
38595,Tia,Charming,Playful,Loyal


No charts were generated by quickchart


In [ ]:
df_name.to_csv(PATH + "/name_adj.csv")